In [ ]:
import math
from copy import deepcopy
import random
import chess
from ChessWrapper import ChessWrapper
import time
import signal
import chess.pgn
from evaluation import *
import pandas as pd
import matplotlib.pyplot as plt
from stockfish import Stockfish
import pickle
import numpy as np
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# load the regression-based static evaluation model
lr_eval = pickle.load(open('lr_eval.pkl', 'rb'))

In [ ]:
importances = lr_eval.coef_
indices = np.argsort(importances)

features = ['tapered_eval', 'king_atk', 'mobility', 'pawn_shield', 'pawn_islands','doubled_pawns', 'passed_pawns']


plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('coefficients')
plt.show()

for i in indices:
    print(importances[i], features[i])

In [ ]:
def reg_eval(board):
    """
    Return a static evaluation of the board using the lr_eval model.
    """
    feat = create_features(board)
    return lr_eval.predict(feat)[0]

In [ ]:
def alphabeta(depth, qdepth, board, alpha, beta, is_max, eval_func=tapered_eval):
    """
    Return the alpha-beta value of a given board with a specified serach depth.

    depth: the depth left to search for alpha-beta.
    qdepth: the qsearch depth.
    board: the current board
    alpha: track white's current best value.
    beta: track black's current best value.
    is_max: True when it's white's turn, False when it's black's turn.
    eval_func: the evaluation function to be used
    """
    if board.is_checkmate():
        return -24000 if is_max else 24000
    # game is over but not checkmate, so it's a draw
    elif board.is_game_over():
        return 0
        
    if depth <= 0:
        qsearch_val = qsearch(qdepth, board, alpha, beta, is_max, eval_func)
        return qsearch_val

    if is_max:
        best_val = float('-inf')
        moves = get_ordered_moves(board, board.get_legal_moves())
        
        for move in moves:
                
            board.push(move)
            v = alphabeta(depth - 1, qdepth, board, alpha, beta, not is_max, eval_func)

            if v > 23000: #checkmate threshold. Incentivize shorter checkmates (ex: M1 better than M5)
                v -= 1
            elif v < -23000:
                v += 1
                
            best_val = max(best_val, v)
            alpha = max(alpha, best_val)
            board.pop()
            if beta <= alpha:
                break
        return best_val
    else:
        best_val = float('+inf')
        moves = get_ordered_moves(board, board.get_legal_moves())
        for move in moves:
            board.push(move)
            v = alphabeta(depth - 1, qdepth, board, alpha, beta, not is_max, eval_func)
            if v > 23000: #checkmate threshold. Incentivize shorter checkmates (ex: M1 better than M5)
                v -= 1
            elif v < -23000:
                v += 1
            best_val = min(best_val, v)
            beta = min(beta, best_val)
            board.pop()
            if beta <= alpha:
                break
        return best_val
        

In [ ]:
def qsearch(qdepth, board, alpha, beta, is_max, eval_func):
    """
    Return the quiescence search value of a given board.

    qdepth: the max depth left to search in quiescence
    board: the current board
    alpha: track white's current best value.
    beta: track black's current best value.
    is_max: True when it's white's turn, False when it's black's turn.
    eval_func: the evaluation function to be used
    """
    if qdepth <= 0:
        return alpha if is_max else beta
    stand_pat = eval_func(board)
    if is_max:
        if stand_pat >= beta:
            return beta
        if stand_pat >= alpha:
            alpha = stand_pat
        moves = board.get_legal_moves()
        captures = [move for move in moves if board.is_capture(move)]
        ordered_captures = get_ordered_captures(board, captures)
        
        for move in ordered_captures:
            board.push(move)
            score = qsearch(qdepth - 1, board, alpha, beta, not is_max, eval_func)
            board.pop()
            if score >= beta:
                return beta
            if score > alpha:
                alpha = score
        return alpha
    else:
        if stand_pat <= alpha:
            return alpha
        if stand_pat <= beta:
            beta = stand_pat
        moves = board.get_legal_moves()
        captures = [move for move in moves if board.is_capture(move)]
        ordered_captures = get_ordered_captures(board, captures)
        
        for move in ordered_captures:
            board.push(move)
            score = qsearch(qdepth - 1, board, alpha, beta, not is_max, eval_func)
            board.pop()
            if score <= alpha:
                return alpha # original: return score?
            if score < beta:
                beta = score
        return beta

In [ ]:
def find_best_move(board, depth, qdepth, eval_func=tapered_eval):
    """
    Given a board, use alpha-beta search (with set depth, qdepth) to find the best move 

    board: the current position to search
    depth: the desired search depth of alpha-beta
    qdepth: the desired max search depth of quiescence
    eval_func: the evaluation function to use
    """
    is_max = (board.get_turn() == chess.WHITE)
    best_val = float('-inf') if is_max else float('+inf')

    if board.is_checkmate():
        return (None, -24000) if is_max else (None, 24000)

    # game is over but not checkmate, so it's a draw
    elif board.is_game_over():
        return (None, 0)
    
    moves = board.get_legal_moves()
    best_move = moves[0]
    for move in moves:
        board.push(move)
        move_val = alphabeta(depth - 1, qdepth, board, float('-inf'), float('+inf'), not is_max, eval_func=eval_func)

        board.pop()
        if is_max and move_val > best_val:
            best_move = move
            best_val = move_val
        elif not is_max and move_val < best_val:
            best_move = move
            best_val = move_val
    return best_move, best_val

In [ ]:
def self_play(depth=4, qdepth=6, eval_func=tapered_eval):
    """
    Have alpha-beta play a game against itself on specified depth, quiescence max depth.
    Return the outcome of the game, and a game object to extract the game's pgn.
    """
    b = ChessWrapper()
    game = chess.pgn.Game()
    game.headers["Event"] = "depth " + str(depth)
    node = None
    wtm = True
    while True:
        if b.is_game_over():
            break
        if wtm:
            m, v = find_best_move(b, depth, qdepth, eval_func=eval_func)
        else:
            m, v = find_best_move(b, depth, qdepth, eval_func=eval_func)

        print('move is: ' + str(b.san(m)) + ' | eval is: ' + str(v))
        print('-' * 15)
        b.push(m)
        wtm = not wtm
        if node is None:
            node = game.add_variation(m)
        else:
            node = node.add_variation(m)
        print(b)
        print('-' * 15)
    return b.outcome(), game

In [ ]:
def vs_person(wtm=True, depth=4, qdepth=6, eval_func=tapered_eval):
    """
    Have alpha-beta play a game against human input.
    """
    
    b = ChessWrapper()
    game = chess.pgn.Game()
    game.headers["Event"] = "vs_person, depth " + str(depth)
    node = None

    while True:
        if b.is_game_over():
            break
        if wtm:
            m, v = find_best_move(b, depth, qdepth, eval_func=eval_func)
        else:
            m = None
            while m is None:
                try:
                    input_move = input('What is the next move? \n')
                    m = b.parse_san(input_move)
                except Exception as e:
                    m = None
            v = None

        print('move is: ' + str(b.san(m)) + ' | eval is: ' + str(v))
        print('-' * 15)
        b.push(m)
        wtm = not wtm
        if node is None:
            node = game.add_variation(m)
        else:
            node = node.add_variation(m)
        print(b)
        print('-' * 15)
    return b.outcome(), game

In [ ]:
# wtm = True means alpha beta moves first, and is white
def vs_sf(elo=1400, depth=4, qdepth=6, wtm=True, fen=None, eval_func=tapered_eval):
    """
    Have alpha-beta play against stockfish, return the outcome and a game object to extract the game's pgn.

    elo: the desired ELO of stockfish
    depth: the depth of alpha-beta search
    qdepth: the max depth of quiescence search
    wtm: True indicates alpha-beta plays first, and plays white, False indicates alpha-beta plays second, and plays black
    fen: FEN string to set up custom starting position
    eval_func: the evaluation function to be used
    """
    if fen is None:
        b = ChessWrapper()

        sf = Stockfish('/opt/homebrew/Cellar/stockfish/16/bin/stockfish')
        sf.set_elo_rating(elo)
    else:
        b = ChessWrapper(fen)
        sf = Stockfish('/opt/homebrew/Cellar/stockfish/16/bin/stockfish')
        sf.set_fen_position(fen)
        sf.set_elo_rating(elo)
    
    game = chess.pgn.Game()
    game.headers["Event"] = "sf elo: " + str(elo) + ', depth: ' + str(depth) + ', qdepth: ' + str(qdepth)
    
    if fen is not None:
        game.setup(fen)
        
    node = None

    while True:
        if b.is_game_over():
            break
        if wtm:
            m, v = find_best_move(b, depth, qdepth, eval_func=eval_func)
        else:
            m, v = chess.Move.from_uci(sf.get_best_move()), None

        b.push(m)
        sf.make_moves_from_current_position([str(m)])
        
        
        
        if node is None:
            node = game.add_variation(m)
        else:
            node = node.add_variation(m)
        
        wtm = not wtm
    return b.outcome(), game

In [ ]:
def est_elo(ab_depth=4, ab_qdepth=6, eval_func=tapered_eval):

    """
    Estimate the ELO of alpha-beta.

    ab_depth: the depth of alpha-beta being tested
    ab_qdepth: the qdepth of alpha-beta being tested
    eval_func: the evaluation function to be used
    """
    
    # sicilian (open), queen's indian, reti transposed to english
    openings = [
        'rnbqkb1r/pp2pppp/3p1n2/8/3NP3/8/PPP2PPP/RNBQKB1R w KQkq - 1 5',
        'rn1qkb1r/p1pp1ppp/bp2pn2/8/2PP4/5NP1/PP2PP1P/RNBQKB1R w KQkq - 1 5',
        'rnbqk2r/ppp1ppbp/3p1np1/8/2P1P3/2N2N2/PP1P1PPP/R1BQKB1R w KQkq - 0 5'
    ]
    
    random.shuffle(openings)
    
    lo = 0
    hi = 3200
    
    df = pd.DataFrame(columns = ['depth', 'qdepth', 'lo', 'hi', 'elo', 'outcome', 'game'])
    
    for i in range(5):
        mid = (lo + hi) // 2
        ab_score = 0
        sf_score = 0
        
        # play 3 pairs
        for i in range(3):
            
            print('-' * 15)
            print(ab_depth, ab_qdepth, mid)
            # alpha beta plays white
            ab_white_oc, ab_white_game = vs_sf(elo=mid, depth=ab_depth, qdepth=ab_qdepth, wtm=True, fen=openings[i], eval_func=eval_func)
            
            if ab_white_oc.winner == chess.WHITE:
                ab_score += 1
            elif ab_white_oc.winner == chess.BLACK:
                sf_score += 1
            else:
                ab_score += 0.5
                sf_score += 0.5
                
            new_row = {'depth': ab_depth, 'qdepth': ab_qdepth, 'lo': lo, 'hi': hi, 'elo': mid, 'outcome': ab_white_oc.winner != chess.BLACK, 'game': ab_white_game}
            df = df._append(new_row, ignore_index=True)
            print(ab_white_oc)
            
            # if either player cuts off early, don't need to play the rest
            if ab_score >= 3 or sf_score >= 3.5:
                break
                
            print('-' * 15)
            print(ab_depth, ab_qdepth, mid)
                
            # alpha beta plays black
            ab_black_oc, ab_black_game = vs_sf(elo=mid, depth=ab_depth, qdepth=ab_qdepth, wtm=False, fen=openings[i], eval_func=eval_func)
            
            if ab_black_oc.winner == chess.BLACK:
                ab_score += 1
            elif ab_black_oc.winner == chess.WHITE:
                sf_score += 1
            else:
                ab_score += 0.5
                sf_score += 0.5
                
            new_row = {'depth': ab_depth, 'qdepth': ab_qdepth, 'lo': lo, 'hi': hi, 'elo': mid, 'outcome': ab_black_oc.winner != chess.WHITE, 'game': ab_black_game}
            df = df._append(new_row, ignore_index=True)
            print(ab_black_oc)
            
            # if either player cuts off early, don't need to play the rest
            if ab_score >= 3 or sf_score >= 3.5:
                break
                
        if ab_score >= 3:
            lo = mid
        else:
            hi = mid
            
    return df, lo, hi

In [ ]:
# fix q search depth, try alpha-beta depth 1, 2, 3, 4
# use tapered evaluation

df_results = pd.DataFrame(columns = ['depth', 'lo', 'hi'])
df_games = pd.DataFrame(columns = ['depth', 'qdepth', 'lo', 'hi', 'elo', 'outcome', 'game'])

for d in range(1, 5):
    df, lo, hi = est_elo(ab_depth=d, ab_qdepth=6, eval_func=tapered_eval)
    new_row = {'depth': d, 'lo': lo, 'hi': hi}
    df_results = df_results._append(new_row, ignore_index=True)
    df_games = pd.concat([df_games, df])
    print(df_results)
    print(df_games)

In [ ]:
# fix q search depth, try alpha-beta depth 1, 2, 3, 4
# use static evaluation with extra features

df_results2 = pd.DataFrame(columns = ['depth', 'lo', 'hi'])
df_games2 = pd.DataFrame(columns = ['depth', 'qdepth', 'lo', 'hi', 'elo', 'outcome', 'game'])

for d in range(1, 5):
    df, lo, hi = est_elo(ab_depth=d, ab_qdepth=6, eval_func=reg_eval)
    new_row = {'depth': d, 'lo': lo, 'hi': hi}
    df_results2 = df_results2._append(new_row, ignore_index=True)
    df_games2 = pd.concat([df_games2, df])
    print(df_results)
    print(df_games)